## **Creating SQL Queries from Natural Language**

This tutorial will guide you through creating an application that generates SQL queries from natural language instructions and evaluates the quality of the generated queries. Along the way, you'll learn how to use Orq's deployment feature to enhance SQL generation. By the end of this tutorial, you'll be ready to experiment with SQL generation in your own projects.

Before starting, ensure you have an Orq account. If not, sign up at Orq.ai. Let's dive in!

Additionally, to simplify the process, we’ve prepared this [Google Colab](https://colab.research.google.com/drive/1OYST2gldxBXbAN10wRTfnTeExCjWrF9i#scrollTo=EJ5-MLEjmpg9) file that you can copy and run immediately after replacing your API key. This file provides a ready-to-use environment with all the required configurations set up, allowing you to focus on experimenting with SQL generation without worrying about initial setup. Let's dive in!

**Step 1: Setting Up the Environment**  
The following commands install the required libraries for working with the Orq platform, handling datasets, and managing the SQL generation workflow. Feel free to reuse and adapt this code for your projects.

## Import packages

In [37]:
pip install orq-ai-sdk datasets huggingface_hub

### Initializing the OrqAI Client

This code initializes the OrqAI client with an API key, either from the `ORQ_API_KEY` environment variable or a hardcoded default, and sets the environment to production.


In [50]:
import os

from orq_ai_sdk import Orq

client = Orq(
  api_key=os.environ.get("ORQ_API_KEY", "YOUR_ORQ_API"),
)

In [48]:
from google.colab import userdata

### **Hugging Face**

Before proceeding, sign up for a free Hugging Face account if you don’t already have one. You’ll need an API key to access their datasets library. Retrieve your API key [here](https://huggingface.co/settings/tokens) after signing up or logging in.

**Step 3: Loading the Dataset**  
Use the Hugging Face datasets library to load a dataset containing table schemas and natural language instructions. Convert the dataset to a pandas DataFrame for easy manipulation.

In [41]:
from huggingface_hub import login
from google.colab import userdata

# Use your Hugging Face API token
login(token="YOUR_HUGGING_FACE_API")

In [42]:
from datasets import load_dataset

ds = load_dataset("Clinton/Text-to-sql-v1")

# Convert to a pandas DataFrame (selecting the "train" split as an example)
df = ds["train"].to_pandas()

# Select the top 50 rows
df = df.head(50)

# Display the DataFrame or save it
print(df)

README.md:   0%|          | 0.00/118 [00:00<?, ?B/s]

texttosqlv2.jsonl:   0%|          | 0.00/635M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/262208 [00:00<?, ? examples/s]

                                          instruction  \
0            Name the home team for carlton away team   
1   what will the population of Asia be when Latin...   
2   How many faculty members do we have for each g...   
3              List the record of 0-1 from the table?   
4   Which silver has a Gold smaller than 12, a Ran...   
5   When did Samsung Electronics Co LTD make the G...   
6   what are the early morning flights from BOSTON...   
7                             Name the most 3 credits   
8   What is every yellow jersey entry for the dist...   
9   In what years was there a rank lower than 9, u...   
10  What aired at 10:00 when Flashpoint aired at 9...   
11  count the number of patients whose insurance i...   
12  What was the record of the game in which Dydek...   
13       When was the game played at glenferrie oval?   
14  What is the highest K 2 O, when Na 2 O is grea...   
15  what is the total number of patients diagnosed...   
16  count the number of patient

In [43]:
df.columns

Index(['instruction', 'input', 'response', 'source', 'text'], dtype='object')

### **SQL Query Generation Use Case**
This deployment is designed to generate valid SQL queries based on specific table schemas and user-provided instructions. The model analyzes the instruction and the associated table schema to produce a precise and contextually appropriate SQL query.

SQL query generation is particularly useful when automating database interactions, building query assistants, or streamlining the process of accessing structured data through natural language inputs.

```plaintext
Below are SQL table schemas paired with instructions that describe a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables:

Here is the instruction: {{instruction}}

Here is the table: {{table}}

OUTPUT ONLY VALID SQL


In [44]:
df = df[["instruction", "input", "response"]]

**Step 4: Generating SQL Queries**

This step involves invoking the Orq deployment to generate SQL queries for each row in the dataset. The instruction column provides the natural language task, while the input column contains the table schema. The results are stored in a new column named output.

In [52]:
# Initialize the outputs list
outputs = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    # Extract the 'instruction' and 'input' columns for each row
    instruction = row["instruction"]
    table = row["input"]

    # Invoke the deployment for each row
    generation = client.deployments.invoke(
        key="text_to_SQL",  # Replace with your actual deployment key
        context={
            "environments": []
        },
        inputs={
            "table": table,
            "instruction": instruction
        },
        metadata={
            "custom-field-name": "custom-metadata-value"
        }
    )

    # Append the model's output to the outputs list
    outputs.append(generation.choices[0].message.content)

# Add the outputs as a new column in the DataFrame
df["output"] = outputs


### Performance Check

**Step 5: Saving and Evaluating Results**  
Save the updated DataFrame containing the SQL queries to a file and evaluate their quality. Use metrics or manual inspection to verify the accuracy and relevance of the generated queries.

In [53]:
df

,instruction,input,response,output
0,Name the home team for carlton away team,CREATE TABLE table_name_77 (\n home_team VA...,SELECT home_team FROM table_name_77 WHERE away...,You are welcome. I am here to help you.
1,what will the population of Asia be when Latin...,"CREATE TABLE table_22767 (\n ""Year"" real,\n...","SELECT ""Asia"" FROM table_22767 WHERE ""Latin Am...",You are welcome. I am here to help.
2,How many faculty members do we have for each g...,"CREATE TABLE Student (\n StuID INTEGER,\n ...","SELECT Sex, COUNT(*) FROM Faculty GROUP BY Sex...",You are welcome. I am here to help.
3,List the record of 0-1 from the table?,CREATE TABLE table_14656147_2 (\n week VARC...,SELECT week FROM table_14656147_2 WHERE record...,You are welcome. I am here to help you.
4,"Which silver has a Gold smaller than 12, a Ran...",CREATE TABLE table_name_24 (\n silver VARCH...,SELECT silver FROM table_name_24 WHERE gold < ...,You are welcome. I am here to help.
5,When did Samsung Electronics Co LTD make the G...,"CREATE TABLE table_47482 (\n ""Company name""...","SELECT ""Date"" FROM table_47482 WHERE ""Company ...",You are welcome. I am here to help.
6,what are the early morning flights from BOSTON...,"CREATE TABLE time_interval (\n period text,...",SELECT DISTINCT flight.flight_id FROM airport_...,You are welcome. I am here to help.
7,Name the most 3 credits,CREATE TABLE table_148535_2 (\n Id VARCHAR\n),SELECT MIN(3 AS _credits) FROM table_148535_2,You are welcome. I am here to help.
8,What is every yellow jersey entry for the dist...,"CREATE TABLE table_3791 (\n ""Year"" text,\n ...","SELECT ""Yellow jersey"" FROM table_3791 WHERE ""...",You are welcome. I am here to help.
9,"In what years was there a rank lower than 9, u...",CREATE TABLE table_name_63 (\n years VARCHA...,SELECT years FROM table_name_63 WHERE matches ...,You are welcome. I am here to help.


In [55]:

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

true_labels = df["response"]
predicted_labels = df["output"]

# Calculate performance metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

# Print the metrics
print("Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Performance Metrics:
Accuracy: 0.0000
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Next Steps**  
Congratulations! You've successfully built and tested a SQL generation application using Orq. To further enhance your project:

- Experiment with different datasets or deployment keys.
- Refine the prompt to improve SQL generation quality.
- Integrate the solution into a larger application for automated data access.

For more details and advanced features, visit the Orq documentation.